In [1]:
import numpy as np
import torch
import torch.nn as nn
import csv
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time

In [2]:
from setup_dataset import *

In [3]:
from setup_model import *

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
"""
data = dataset(match_list=[1, 2, 3, 4, 5],
               start_frame = 1,
               end_frame = 1020
              )
data.data
"""

'\ndata = dataset(match_list=[1, 2, 3, 4, 5],\n               start_frame = 1,\n               end_frame = 1020\n              )\ndata.data\n'

In [6]:
data = chunk_dataset(match_list=[1],
                     start_frame = 1,
                     end_frame = 1020
                    )

Successfully loaded NPZ.
Do you want to use GPU (y/n)n
Selected CPU


C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:139: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train = np.array(X_train)
C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:140: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  Y_train = np.array(Y_train)


In [7]:
len(data.data)

124

In [8]:
data.data[0].shape

torch.Size([29, 100, 120])

In [9]:
data.data

[tensor([[[0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          ...,
          [0.1569, 0.1569, 0.1569,  ..., 0.7529, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569]],
 
         [[0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          ...,
          [0.1569, 0.1569, 0.1569,  ..., 0.7529, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.7529, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.7529, 0.1569]],
 
         [[0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
          [0.1569, 0.1569, 0.1569,  ...,

In [10]:
data.padSequence()

In [11]:
data.data.shape

torch.Size([124, 30, 100, 120])

In [12]:
original_shape = data.data.shape
data.data = data.data.reshape(original_shape[0], original_shape[1], 12000)

In [13]:
data.data.shape

torch.Size([124, 30, 12000])

In [14]:
data.data.shape, data.targets.shape, data.seq_len.shape

(torch.Size([124, 30, 12000]), torch.Size([124, 30, 9]), torch.Size([124]))

In [15]:
device = data.device
input_size = data.data.shape[2]
output_size = len(data.available_targets)
match_list = data.match_list
start_frame = data.start_frame
end_frame = data.end_frame

In [16]:
data.packPaddedSequence()

In [17]:
data.data[0].shape

torch.Size([1018, 12000])

In [18]:
model_structure = ModelStructure(device, input_size, output_size, match_list, start_frame, end_frame)

Number of hidden neurons: 200
Number of epochs: 5500
Choose type of RNN model:
1 - Simple RNN
2 - LSTM
3 - CNN
type: 2
write a observations without space and punctuations:teste
models/LSTM_teste_m1_f1to1020_epoch5500_h200
ATTENTION! folder not created. Training informations will overwrite the existing one


In [19]:
def set_model(model_structure):

    if model_structure.type == "RNN":
        model = RNNModel(device=model_structure.device, 
                         input_size=model_structure.data_size, 
                         output_size=model_structure.output_size, 
                         hidden_dim=model_structure.hidden_neurons, 
                         n_layers=1)
    elif model_structure.type == "LSTM":
        model = LSTMModel(device=model_structure.device, 
                          input_size=model_structure.data_size, 
                          output_size=model_structure.output_size, 
                          hidden_dim=model_structure.hidden_neurons, 
                          n_layers=1)
    elif model_structure.type == "CNN":
        model = CNNLSTMModel(device=model_structure.device, 
                             input_size=model_structure.data_size, 
                             output_size=model_structure.output_size, 
                             hidden_dim=model_structure.hidden_neurons, 
                             n_layers=1)
    else:
        print("ERROR defining Model")

    return model

In [20]:
criterion = nn.MSELoss()
model = set_model(model_structure)
optimizer = torch.optim.Adam(model.parameters())

In [21]:
if device.type == 'cuda':
    mymodel = model.cuda()
    X_train = data.data.cuda() 
    Y_train = data.targets.cuda()
else:
    mymodel = model
    X_train = data.data 
    Y_train = data.targets

In [22]:
def get_acc(predicted, target):
    
    predicted = torch.argmax(predicted, axis=1)
    target = torch.argmax(target, axis=1)

    acc = np.mean(predicted == target)
    
    return float(acc)

In [23]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])

In [24]:
X_train

PackedSequence(data=tensor([[0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        ...,
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.7529, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569]]), batch_sizes=tensor([124, 119, 111,  92,  77,  71,  60,  52,  46,  40,  35,  29,  26,  24,
         19,  18,  15,  14,  11,   6,   4,   4,   4,   4,   3,   3,   2,   2,
          2,   1]), sorted_indices=tensor([ 13,   0,   5,  43,   7,  19, 106,  12, 114,  32, 116,   4,  47,  45,
         74,   1,  97,  90,  88,   9,  56,  34,  80,  95,  49,  76,  58,  41,
         53,  21,  14,   6, 100,  96, 113,  75,  72,  17,   2,  55,  83,  71,
         86,  15,  92,  40, 109, 118,  29, 119,  77,  68,   3,  11,  16,  18,
         79,  26, 117,  63,  93,  85, 104,  81,  65, 101, 112,  42

In [25]:
X_train

PackedSequence(data=tensor([[0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        ...,
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.7529, 0.1569],
        [0.1569, 0.1569, 0.1569,  ..., 0.1569, 0.1569, 0.1569]]), batch_sizes=tensor([124, 119, 111,  92,  77,  71,  60,  52,  46,  40,  35,  29,  26,  24,
         19,  18,  15,  14,  11,   6,   4,   4,   4,   4,   3,   3,   2,   2,
          2,   1]), sorted_indices=tensor([ 13,   0,   5,  43,   7,  19, 106,  12, 114,  32, 116,   4,  47,  45,
         74,   1,  97,  90,  88,   9,  56,  34,  80,  95,  49,  76,  58,  41,
         53,  21,  14,   6, 100,  96, 113,  75,  72,  17,   2,  55,  83,  71,
         86,  15,  92,  40, 109, 118,  29, 119,  77,  68,   3,  11,  16,  18,
         79,  26, 117,  63,  93,  85, 104,  81,  65, 101, 112,  42

In [26]:
start_time_processing = time.time()

loss_file = open(model_structure.path + '/' + "loss_file.txt", "w")
first_time = True

best_loss = 1
first_epoch = True

for epoch in range(1, model_structure.n_epochs + 1):

    mymodel.train()

    optimizer.zero_grad()
    output = mymodel(X_train)
    
    loss = criterion(output, Y_train.view(-1,data.target_size[2]).float())
    loss.backward()
    optimizer.step()
        
    if epoch%10 == 0:

        train_loss_arr = np.append(train_loss_arr, loss.item())
        train_acc_arr  = np.append(train_acc_arr, model_structure.get_acc(output, Y_train.reshape(-1, len(data.available_targets))))
        
        loss_file.write("Epoch: {}/{}-------------------------------------------\n".format(epoch, model_structure.n_epochs))
        loss_file.write("Train -> Loss: {:.15f} Acc: {:.15f}\n".format(train_loss_arr[-1], train_acc_arr[-1]))
            
        print("Epoch: {}/{}-------------------------------------------".format(epoch, model_structure.n_epochs))
        print("Train -> Loss: {:.15f} Acc: {:.15f}".format(train_loss_arr[-1], train_acc_arr[-1]))
        
        if train_loss_arr[-1] < best_loss:
            state = { 'epoch': epoch + 1, 'state_dict': mymodel.state_dict(),
                      'optimizer': optimizer.state_dict(), 'losslogger': loss.item(), }
            torch.save(state, model_structure.path + '/' + model_structure.name)
            best_loss = loss.item()
        else:
            print("model not saved")
            
loss_file.write("--- %s seconds ---" % (time.time() - start_time_processing))
loss_file.close()
np.savez(model_structure.path + '/' + "train_loss_arr", train_loss_arr)
print("--- %s seconds ---" % (time.time() - start_time_processing))

C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_model_types.py:76: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.out(out)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
plt.plot(train_loss_arr, color='blue')
plt.title('model train loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.yscale('log')
plt.savefig(model_structure.path + '/' + 'train_loss.png')